#Install

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install dipy

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import collections
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import VectorOfEndpointsFeature
from dipy.segment.clustering import QuickBundles

#Route similarity

In [15]:
trips_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel'

In [16]:
road_data_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/route_network/clustered'

In [32]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel'

trips_2019_1_29.csv  trips_2019_3_30.csv  trips_2019_6_9.csv
trips_2019_1_30.csv  trips_2019_4_23.csv  trips_2019_7_3.csv
trips_2019_2_18.csv  trips_2019_4_24.csv  trips_2019_7_4.csv
trips_2019_2_25.csv  trips_2019_5_18.csv  trips_2019_7_5.csv
trips_2019_3_24.csv  trips_2019_5_20.csv  trips_2019_7_6.csv
trips_2019_3_27.csv  trips_2019_6_7.csv


In [30]:
dfs = []
day_len = []
day_trip_count = []
months = range(1,8)
days = range(1,32)
for month in months:
  for day in days:
    try:
      sp_seq_df = pd.read_csv(trips_path + f'/trips_2019_{month}_{day}.csv', parse_dates=['time_stamp'])
      print(len(sp_seq_df))
      print(trips_path + f'/trips_2019_{month}_{day}.csv')
    except:
      pass
    
    sp_seq_df = sp_seq_df[sp_seq_df['cluster']>-1].sort_values(by=['trip_id', 'time_stamp'])

    trip_count = sp_seq_df.trip_id.value_counts()
    trip_count = trip_count[trip_count >= 12]
    sp_seq_df = sp_seq_df[sp_seq_df.trip_id.isin(trip_count.index)]
    sp_seq_df['month'] = month
    sp_seq_df['day'] = day
    day_len.append(len(sp_seq_df))
    day_trip_count.append(len(trip_count))
    dfs.append(sp_seq_df)
print(day_len)
print(day_trip_count)
df = pd.concat(dfs, ignore_index=True)
df

492231
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_1_29.csv
497319
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_1_30.csv
513950
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_2_18.csv
516362
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_2_25.csv
411779
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_3_24.csv
284415
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_3_27.csv
345748
/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/trips_for_lebel/trips_2019_3_30.csv
419446
/content/drive/My Drive/2021 Route

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,cluster,month,day
0,2019-07-06 04:27:55,091000100000000001030212027,14.895970,101.892716,58,7.0,8,96,1,1
1,2019-07-06 04:28:00,091000100000000001030212027,14.896270,101.893416,59,7.0,8,96,1,1
2,2019-07-06 04:37:00,091000100000000001030212027,14.925045,101.961618,54,7.0,8,168,1,1
3,2019-07-06 04:40:00,091000100000000001030212027,14.934837,101.984593,47,7.0,8,166,1,1
4,2019-07-06 04:40:05,091000100000000001030212027,14.935072,101.985146,46,7.0,8,166,1,1
...,...,...,...,...,...,...,...,...,...,...
414332,2019-07-06 09:57:34,0030007000002G02A7BPA800033,14.841046,101.757140,73,7.0,2412,912,7,31
414333,2019-07-06 12:26:34,0030007000002G02A7BPA800033,14.607074,103.120586,70,7.0,2412,355,7,31
414334,2019-07-06 14:02:34,0030007000002G02A7BPA800033,14.637503,103.991504,74,7.0,2412,1142,7,31
414335,2019-07-06 14:06:34,0030007000002G02A7BPA800033,14.642861,104.036431,75,7.0,2412,1140,7,31


In [33]:
road_df = pd.read_csv(road_data_path+"/route_network_with_cluster.csv")

cluster_df = road_df[['cluster', 'center_lat', 'center_lon']]
cluster_df = cluster_df.drop_duplicates()
cluster_df.rename(columns = {'cluster':'sp', 'center_lat':'lat', 'center_lon':'lon'}, inplace = True)
cluster_df

,sp,lat,lon
0,0,14.984788,102.051257
156,1,14.979075,102.020065
188,2,14.982676,102.003040
220,3,14.987172,101.986551
244,4,14.992287,101.968000
...,...,...,...
50516,8049,15.765768,101.555882
50518,8050,16.132595,101.584201
50522,8051,15.755759,101.582307
50524,8052,15.557630,101.573573


In [34]:
df.rename(columns = {'cluster':'sp'}, inplace = True)
loc_df = df[['trip_id', 'sp', 'month','day']].merge(cluster_df,how='left')
print(sp_seq_df.isna().any())
loc_df

time_stamp    False
unit_id       False
lat           False
lon           False
speed         False
unit_type     False
trip_id       False
cluster       False
month         False
day           False
dtype: bool


,trip_id,sp,month,day,lat,lon
0,8,96,1,1,14.895917,101.892895
1,8,96,1,1,14.895917,101.892895
2,8,168,1,1,14.925277,101.962362
3,8,166,1,1,14.934784,101.984801
4,8,166,1,1,14.934784,101.984801
...,...,...,...,...,...,...
414332,2412,912,7,31,14.841173,101.756559
414333,2412,355,7,31,14.606764,103.121223
414334,2412,1142,7,31,14.637615,103.991864
414335,2412,1140,7,31,14.642905,104.036175


In [35]:
trips = []
low_12 = 0
for month in months:
  for day in days:
    day_df = loc_df[(loc_df.day == day) & (loc_df.month == month)]
    for t_id in day_df.trip_id.unique():
      trip = day_df[day_df.trip_id == t_id][['lat', 'lon']]
      if len(trip)/12 > 1.5:
        split_trip=[trip.iloc[i:i + 12] for i in range(0, len(trip), 12)]
        if len(split_trip[-1]) < 8:
          last = split_trip.pop()
          before_last = split_trip.pop()
          merge_last = pd.concat([before_last, last])
          split_trip.append(merge_last)
        for partial_trip in split_trip:
          lat_lon_data = np.c_[partial_trip['lat'].values, partial_trip['lon'].values]
          trips.append(lat_lon_data)
      else:
        # print(len(trip))
        lat_lon_data = np.c_[trip['lat'].values, trip['lon'].values]
        trips.append(lat_lon_data)
# print(low_12)
print(len(trips))
trips[0][0,0], trips[0][0,1]

31496


(14.8959166, 101.89289514999996)

In [36]:
train_path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/model/3.1'

In [ ]:
import pickle
import collections
import geopy.distance
from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from dipy.segment.metric import VectorOfEndpointsFeature
from dipy.segment.clustering import QuickBundles
class GPSDistance(Metric):
    def __init__(self):
        super(GPSDistance, self).__init__(feature=ResampleFeature(nb_points=12))

    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)

    def dist(self, v1, v2):
        x = [geopy.distance.distance([p[0][0], p[0][1]], [p[1][0], p[1][1]]).kilometers for p in list(zip(v1, v2))]
        currD = np.mean(x)
        return currD

THRESHOLD = 12
metric = GPSDistance()
qb = QuickBundles(threshold=THRESHOLD, metric=metric)

# feature = ResampleFeature(nb_points=24)
# metric = AveragePointwiseEuclideanMetric(feature=feature)
# qb = QuickBundles(threshold=THRESHOLD, metric=metric)

%time clusters = qb.cluster(trips)

pkl_filename = train_path+"/QBcluster_baseline_treshold12.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clusters, file)
with open(pkl_filename, 'rb') as file:
    clusters = pickle.load(file)

print("Nb. clusters:", len(clusters))
print("Cluster sizes:", map(len, clusters))
print("Small clusters:", collections.Counter(clusters < 10)[True], end=' /')
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)